In [347]:
from enum import Enum, StrEnum
import torch
import torch.nn as nn
from torch.nn import functional as F

class DataType(Enum):
    TRAIN = "TRAIN"
    VALIDATE = "VALIDATE"

class DeviceType(StrEnum):
    CUDA = "cuda"
    CPU = "cpu"

# Globals
g_batch_size = 64 # How many independent sequences in parallel
g_block_size = 256 # Maximum context length for predictions
max_iterations = 5000
eval_interval = 500
learning_rate = 3e-4
device = DeviceType.CUDA if torch.cuda.is_available() else DeviceType.CPU
eval_iterations = 200
g_num_embedding_dim = 384
g_num_heads = 6
g_num_layers = 6
g_dropout = 0.2

# Seed Torch
torch.manual_seed(13377)

In [348]:
# Get our data set
with open("tinyshakespeare.txt", encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

In [349]:
# Tokenize our data
str_to_int = {c:i for i,c in enumerate(chars)}
int_to_str = {v:k for k,v in str_to_int.items()}

def encode(seq: str) -> list[int]:
    return [str_to_int[c] for c in seq]

def decode(seq: list[int]) -> str:
    return ''.join([int_to_str[l] for l in seq])

In [350]:
# Split up data into train and validation sets
data = torch.tensor(encode(text), dtype=torch.long)
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

In [351]:
def get_batch(split: DataType) -> tuple[torch.Tensor, torch.Tensor]:
    data = train_data if split == DataType.TRAIN else val_data
    ix = torch.randint(len(data) - g_block_size, (g_batch_size,))
    x = torch.stack([data[i:i+g_block_size] for i in ix])
    y = torch.stack([data[i+1:i+g_block_size+1] for i in ix])

    return x.to(device), y.to(device)

In [352]:
@torch.no_grad()
def estimate_loss(model: nn.Module) -> dict[DataType,torch.Tensor]:
    out: dict[DataType,torch.Tensor] = {}
    model.eval()
    for split in DataType:
        losses = torch.zeros(eval_iterations)
        for k in range(eval_iterations):
            X, Y = get_batch(split)
            _, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()

    return out

In [353]:
class Head(nn.Module):
    def __init__(
        self, head_size: int, block_size: int, embedding_dim: int, dropout: float
    ) -> None:
        super().__init__()
        self.key = nn.Linear(embedding_dim, head_size, bias=False)
        self.query = nn.Linear(embedding_dim, head_size, bias=False)
        self.value = nn.Linear(embedding_dim, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(
        self, x: torch.Tensor | None = None
    ) -> tuple[torch.Tensor, torch.Tensor | None]:
        batch, time, channels = x.shape
        k = self.key(x)  # (Batch, Time, Channels)
        q = self.query(x)  # (Batch, Time, Channels)

        # compute attention scores ("affinities")
        # (B, T, C) @ (B, C, T) => (B, T, T)
        weight = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
        weight = weight.masked_fill(self.tril[:time, :time] == 0, float("-inf"))
        weight = F.softmax(weight, dim=-1)  # (Batch, Time, Time)
        weight = self.dropout(weight)
        # performed the weighted aggregation of the values
        v = self.value(x)  # (Batch, Time, Channels)

        return weight @ v  # (B, T, T) @ (B, T, C) => (B, T, C)


In [354]:
class MultiHeadAttention(nn.Module):
    """Multiple heads of self-attention in parallel"""

    def __init__(
        self,
        num_heads: int,
        head_size: int,
        block_size: int,
        embedding_dim: int,
        dropout: float,
    ) -> None:
        super().__init__()
        self.heads = nn.ModuleList(
            [
                Head(
                    head_size=head_size,
                    block_size=block_size,
                    embedding_dim=embedding_dim,
                    dropout=dropout,
                )
                for _ in range(num_heads)
            ]
        )
        self.proj = nn.Linear(head_size * num_heads, embedding_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


In [355]:
class FeedFoward(nn.Module):
    """A simple linear layer followed by a non-linearity"""

    def __init__(self, embedding_dim: int, dropout: float) -> None:
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embedding_dim, 4 * embedding_dim),
            nn.ReLU(),
            nn.Linear(4 * embedding_dim, embedding_dim),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


In [356]:
class Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(
        self, embedding_dim: int, num_heads: int, block_size: int, dropout: float
    ) -> None:
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        self.self_attention = MultiHeadAttention(
            num_heads=num_heads,
            head_size=embedding_dim // num_heads,
            block_size=block_size,
            embedding_dim=embedding_dim,
            dropout=dropout,
        )
        self.feed_forward = FeedFoward(embedding_dim=embedding_dim, dropout=dropout)
        self.layer_norm_1 = nn.LayerNorm(embedding_dim)
        self.layer_norm_2 = nn.LayerNorm(embedding_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x + self.self_attention(self.layer_norm_1(x))
        x = x + self.feed_forward(self.layer_norm_2(x))
    
        return x


In [357]:
class GPTLanguageModel(nn.Module):
    def __init__(
        self,
        num_embeddings: int,
        block_size: int,
        embedding_dim: int,
        dropout: float,
        num_heads: int,
        num_layers: int
    ) -> None:
        super().__init__()
        self.block_size = block_size
        self.token_embedding_table = nn.Embedding(num_embeddings, embedding_dim)
        self.position_embedding_table = nn.Embedding(block_size, embedding_dim)
        self.blocks = nn.Sequential(*[Block(
            embedding_dim=embedding_dim,
            num_heads=num_heads,
            block_size=block_size,
            dropout=dropout,
        ) for _ in range(num_layers)])
        self.ln_f = nn.LayerNorm(embedding_dim)  # final layer norm
        self.lm_head = nn.Linear(embedding_dim, num_embeddings)

        self.apply(self._init_weights)

    def _init_weights(self, module: nn.Module) -> None:
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(
        self, idx: torch.Tensor, targets: torch.Tensor | None = None
    ) -> tuple[torch.Tensor, torch.Tensor | None]:
        batch, time = idx.shape

        # Compute scores for next char in sequence
        token_embedded = self.token_embedding_table(idx)  # (Batch, Time, Channel)
        postion_embedded = self.position_embedding_table(
            torch.arange(time, device=device)  # (Time, Channel)
        )
        x = token_embedded * postion_embedded  # (Batch, Time, Channel)
        x = self.blocks(x)  # apply one head of self-attention. (B, T, C)
        x = self.ln_f(x)
        logits = self.lm_head(x)  # (Batch, Time, Num Embeddings)

        # Reshape logits into a 2 dimension shape
        loss: torch.Tensor | None = None
        if targets is not None:
            batch, time, channel = logits.shape
            logits = logits.view(batch * time, channel)
            targets = targets.view(batch * time)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx: torch.Tensor, max_new_tokens: int) -> torch.Tensor:
        for _ in range(max_new_tokens):
            logits, _ = self(idx[:, -self.block_size :])  # get predictions
            logits = logits[:, -1, :]  # (Batch, Channel) for last time
            # Use softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (Batch, Channel)
            # Sample from the probability distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (Batch, 1)
            # Append the sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (Batch, Time+1)

        return idx


In [358]:
# NLP Model
model = GPTLanguageModel(
    num_embeddings=vocab_size,
    block_size=g_block_size,
    embedding_dim=g_num_embedding_dim,
    dropout=g_dropout,
    num_heads=g_num_heads,
    num_layers=g_num_layers
)
m = model.to(device)

# Create a PyTorch optimizer
# Using adam here, gradient descent would be the simpliest thing we can use
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)


In [359]:
# Train and reduce loss
first_loss: float | None = None
for iter in range(max_iterations):
    # Evaluate loss every not and then
    if iter % eval_interval == 0:
        losses = estimate_loss(model)
        print(f"Stop {iter}:"
              f"\tTrain loss: {losses[DataType.TRAIN]}"
              f"\tValidation loss: {losses[DataType.VALIDATE]}")

    # sample a batch of data
    xb, yb = get_batch(DataType.TRAIN)

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Generate the Model
idx = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(idx, max_new_tokens=500)[0].tolist()))

In [ ]:
# Generate the Model
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))



MA'ly rirt gard Ding not retl!
KOKen I whach ee brome fhoullaced, centur
Biels the Tie;
Pus i?

Ond ruf faisgolby, mar?

KHhicg,.

You yomos hel isghursce whapen whatti:
I nowrie che-bre
In Ge len haisspespetatg drut theerset me
E Ond anou inetant.
CORIY:
Eslvourse sast othee goatichin to omt gly: credd ase thasen thom;
Dake he wiipt cand lowiil Arov yer wetinge so gors phace;
Weer me!
Amave os!
Af il of oroot cralizenmere lisownocs bwil, my opmiaod wripot boiois wol.

Caiome meut YI I hasunt V
